In [ ]:
# connect to postgresql db usingenvironment variable read from vars.env 

import warnings
warnings.filterwarnings("ignore")

import os
import psycopg2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# read environment variables from vars.env
from dotenv import load_dotenv
load_dotenv("../vars.env")

# connect to postgresql db on localhost, post 5432, using user and password from vars.env

import psycopg2
import os

# Define the database credentials
db_host = os.getenv("POSTGRES_HOST")
db_name = os.getenv("POSTGRES_DB")
db_user = os.getenv("POSTGRES_USER")
db_password = os.getenv("POSTGRES_PWD")

# Create the connection
connection = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password
)



In [ ]:
# remove folders under/data/tts-qa/share_*'that starts with share 

# !rm -rf /data/tts-qa/share_* # be careful with this command, it will remove all folders under /data/tts-qa/share_* 

In [ ]:
total_hours = 30

In [ ]:
stats = {}
for dataset_str in ["German(Dorothee)", "French(Dorsaf)", "English(Melynda)",  "Italian(Martina)", "Spanish(Violeta)"]:
    # Define the path to the SQL scriptorder by sample.wer desc 
    query = f"""
    SELECT sample.* 
    FROM sample
    JOIN dataset ON sample.dataset_id = dataset.id
    WHERE dataset.name='{dataset_str}' 
        AND sample.custom_trimmed_audio_duration > 0 
        AND sample.custom_trimmed_audio_duration < 5 
        AND sample.longest_pause < 0.5
    """

    

    # Assume you've loaded the sample and annotation tables as DataFrames
    sample_df = pd.read_sql(query, connection)
    annotation_df = pd.read_sql("SELECT * FROM annotation", connection)

    # Merge the DataFrames based on the sample.id and annotation.sample_id
    merged_df = pd.merge(sample_df, annotation_df, left_on='id', right_on='sample_id', how='left')

    # discard th discarded
    merged_df = merged_df[merged_df['status'] != 'Discarded']

    # drop  duplicates in id_x and keep the first one
    merged_df.drop_duplicates(subset=['id_x'], keep='first', inplace=True)

    #rename idx to id
    merged_df.rename(columns={'id_x': 'id'}, inplace=True)


    # Compute the required columns
    merged_df['words_per_second'] = merged_df['sentence_length'] / merged_df['custom_trimmed_audio_duration']
    merged_df['text'] = merged_df['final_text'].combine_first(merged_df['original_text'])
    merged_df['wer'] = merged_df.apply(lambda row: (row['wer'] + row['uncased_unpunctuated_wer'])/2 if pd.isnull(row['final_text']) else 0, axis=1)
    merged_df['wer_wo_punctuation'] = merged_df.apply(lambda row: row['uncased_unpunctuated_wer'] if pd.isnull(row['final_text']) else 0, axis=1)
    merged_df['has_annotation'] = ~merged_df['final_text'].isnull()
    merged_df['duration'] = merged_df['custom_trimmed_audio_duration']

    # Select the columns you want in the final result
    df = merged_df[['id', 'filename', 'local_custom_trimmed_path', 'words_per_second', 'text', 'wer', 'wer_wo_punctuation', 'has_annotation',  'duration']]


    # Optionally, you can sort by the has_annotation column
    df = df.sort_values(by='has_annotation', ascending=False)


    # Execute the SQL script into pandas dataframe with column names
    df.sort_values(by=['wer'], inplace=True, ascending=True)
    df = df[df['wer'] <=0.5]
    df.dropna(inplace=True)


    from numpy import percentile
    wps = df['words_per_second'].values

    # calculate interquartile range
    q25, q75 = percentile(wps, 25), percentile(wps, 75)
    iqr = q75 - q25

    # calculate the outlier cutoff
    cut_off = iqr * 1.25
    lower, upper = q25 - cut_off, q75 + cut_off

    print(f"Lower: {lower}, Upper: {upper}")
    # identify outliers
    df["is_outlier"] = (df['words_per_second'] < lower) | (df['words_per_second'] > upper)
    outliers = df[df['is_outlier'] == True]
    print('Identified outliers: %d' % len(outliers))

    df = df[df['is_outlier'] == False]
    df.drop(['is_outlier'], axis=1, inplace=True)
    
    

    
    # create a language folder under it 
    if "English" in dataset_str:
        dataset = "English"
    elif "Spanish" in dataset_str:
        dataset = "Spanish"
    elif "German" in dataset_str:
        dataset = "German"
    elif "French" in dataset_str:
        dataset = "French"
    elif "Italian" in dataset_str:
        dataset = "Italian"

    df_share_all = df[df.wer_wo_punctuation ==0]
    if df_share_all['duration'].sum() / 60 / 60 < total_hours:
        cutoff_idx = df['duration'].cumsum().searchsorted(total_hours*60*60)
        df_share_wavs = df.iloc[:cutoff_idx]
        # empty df
        df_share_extras = pd.DataFrame(columns=df.columns)
    else:
        cutoff_idx = df_share_all['duration'].cumsum().searchsorted(total_hours*60*60)
        df_share_wavs = df_share_all.iloc[:cutoff_idx]
        df_share_extras = df_share_all.iloc[cutoff_idx:]

   

    # make a share folder
    share_folder = f"/data/tts-qa/share_{total_hours}h"
    if not os.path.exists(share_folder):
        os.mkdir(share_folder)


    lang_folder = os.path.join(share_folder, dataset)
    if not os.path.exists(lang_folder):
        os.mkdir(lang_folder)

    # create "wav" folder under the language folder
    wav_folder = os.path.join(lang_folder, "wavs")
    if not os.path.exists(wav_folder):
        os.mkdir(wav_folder)
    
    extras_folder = os.path.join(lang_folder, "extras")
    if not os.path.exists(extras_folder):
        os.mkdir(extras_folder)

    # copy audio files to the language folder
    import shutil
    from tqdm import tqdm
    df_share_all_concat = []
    for df_share, folder in [(df_share_wavs, 'wavs'), (df_share_extras, 'extras')]:
        if folder == 'wavs':
            to_folder = wav_folder
        else:
            to_folder = extras_folder
        for index, row in  tqdm(df_share.iterrows(), total=df_share.shape[0]):
            shutil.copy(row['local_custom_trimmed_path'], to_folder)
        df_share["filepath"] = df_share["filename"].apply(lambda x: os.path.join(folder, x))
        df_share_all_concat.append(df_share)
    
    df_share = pd.concat(df_share_all_concat)
    # drop the local_custom_trimmed_path  and wer, id columns
    df_share_clean = df_share.drop(['local_custom_trimmed_path','words_per_second', 'id'], axis=1)
    # create a csv file with the same name as the language folder
    # sort on by filename
    df_share_clean = df_share_clean[['filename','filepath', 'text', 'duration', 'wer','wer_wo_punctuation']]
    # df_share_clean.to_csv(os.path.join(lang_folder, dataset + ".csv"), index=False)
    # save wavs and extras separately
    df_share_clean_wavs = df_share_clean[~df_share_clean['filepath'].str.contains("extras")]
    df_share_clean_wavs.to_csv(os.path.join(lang_folder, f"{dataset}.csv"), index=False)
    df_share_clean_extras = df_share_clean[df_share_clean['filepath'].str.contains("extras")]
    df_share_clean_extras.to_csv(os.path.join(lang_folder, f"{dataset}-extras.csv"), index=False)
    
    stats[f"{dataset}"] = {
        "total_duration (hours)": df_share_clean_wavs['duration'].sum() / 60 / 60,
        "file count" : df_share_clean_wavs.shape[0],
        "wer=0 (percentage)": df_share_clean_wavs[df_share_clean_wavs['wer'] == 0].shape[0] / df_share_clean_wavs.shape[0],
        "avg wer (wer!=0)": df_share_clean_wavs[df_share_clean_wavs['wer'] != 0]['wer'].mean(),
    }

    if df_share_clean_extras.shape[0] > 0:
        stats[f"{dataset}-extras"] = {
            "total_duration (hours)": df_share_clean_extras['duration'].sum() / 60 / 60,
            "file count" : df_share_clean_extras.shape[0],
            "wer=0 (percentage)": df_share_clean_extras[df_share_clean_extras['wer'] == 0].shape[0] / df_share_clean_extras.shape[0],
            "avg wer (wer!=0)": df_share_clean_extras[df_share_clean_extras['wer'] != 0]['wer'].mean(),
        }

    print(f"Total duration: {df_share_clean['duration'].sum() / 60 / 60} hours")


In [ ]:
pd.DataFrame(stats)


In [ ]:
df_share_clean_wavs

In [ ]:
import pandas as pd
import shutil
from tqdm import tqdm

for dataset in ["German", "French", "English", "Italian", "Spanish"]:
    # for each of the dataset generate a random 100 sample and save it to a csv file as well as the audio files
    sample_folder = "sample_100"
    if not os.path.exists(os.path.join(sample_folder, dataset)):
        os.makedirs(os.path.join(sample_folder, dataset))

    if not os.path.exists(os.path.join(sample_folder, dataset, "wavs")):
        os.makedirs(os.path.join(sample_folder, dataset, "wavs"))
        
    df = pd.read_csv(f"/data/tts-qa/share_{total_hours}h/{dataset}/{dataset}.csv")


    df_sample = df.sample(n=100, random_state=1)
    df_sample.to_csv(f"{sample_folder}/{dataset}/{dataset}.csv", index=False)

    # copy audio files to the language folder
    for index, row in  tqdm(df_sample.iterrows(), total=df_sample.shape[0]):
        shutil.copy(f"/data/tts-qa/share_{total_hours}h/{dataset}/{row['filepath']}", os.path.join(sample_folder, dataset, "wavs"))



In [ ]:
# zip the sample_100 folder
import shutil

shutil.make_archive('sample_100', 'zip', 'sample_100')


In [ ]:
df.sort_values(by=['wer'], inplace=True, ascending=True)

In [ ]:
df

In [ ]:
df.wer.plot.hist(bins=100)

## Create Bash Script for Sharing Data

In [ ]:
# write down a bash script to go the /data/tts-qa and zip all the folders in a for loop 
# then upload the zip file to s3://user-ahmet/translated-Spanish-Italian-French-10h.zip

with open(os.path.join('/data/tts-qa', "zip.sh"), "w") as f:
    f.write("#!/bin/bash\n")
    f.write(f"cd /data/tts-qa/share_{total_hours}h\n")
    
    f.write("for d in */ ; do\n")
    # add logging 
    f.write("    echo \"zipping $d\"\n")
    # get the name 
    f.write("    folder=${d%/}\n")
    f.write("    zip -r $folder.zip $d\n")
    f.write("done\n")
    # include ony zip
    f.write(f"aws s3 cp --recursive  --exclude \"*\" --include \"*.zip\" ./ s3://user-ahmet/Translated-{total_hours}h-v2/\n")
    f.write("echo \"done\"\n")


In [ ]:
# loisten audio
import IPython.display as ipd


index = -3
print(df_share.iloc[index]["filename"])
print(df_share.iloc[index]["text"])
ipd.Audio(df_share.iloc[index]["local_custom_trimmed_path"])


In [ ]:
ipd.Audio(df_share.iloc[index]["local_path"])

In [ ]:
df_share.tail(10)

In [ ]:
# # write a bash script to get dump of a database for every hour and upload to s3 
# # commands looks like following
# # docker exec -t postgres_container_dev pg_dump -U postgres  dev_tts_db > /data/tts-qa/dumps/dump_`date +%Y-%m-%d"_"%H_%M_%S`.sql
# # aws s3 sync /data/tts-qa/dumps/ s3://user-ahmet/tts-qa-dumps/
# import os
# with open(os.path.join('..', "dump.sh"), "w") as f:
#     f.write("#!/bin/bash\n")
#     f.write(f"cd /data/tts-qa/dumps\n")
#     f.write("docker exec -t postgres_container_dev pg_dump -U postgres  dev_tts_db > /data/tts-qa/dumps/dump_`date +%Y-%m-%d\"_\"%H_%M_%S`.sql\n")
#     f.write("aws s3 sync /data/tts-qa/dumps/ s3://user-ahmet/tts-qa-dumps/\n")
#     f.write("echo \"done\"\n")

In [ ]:
# write cron job to run every hour
# crontab -e
# 0 * * * * /data/tts-qa/dump.sh